In [ ]:
from app.audio_processor import AudioProcessor
from app.data_retrival import DataRetrieval
from app.data_validator import DataValidator
from IPython.display import Audio
from pathlib import Path
import json
from tqdm import tqdm

In [ ]:
audio_processor=AudioProcessor()
data_retrieval=DataRetrieval()
data_validator=DataValidator()

In [ ]:
name="DJNsknmKrwM"

In [ ]:
all_audio=list(Path(f"/home/munikumar-17774/Desktop/data/data/{name}").glob("*.mp3"))

In [ ]:
with open(f"/home/munikumar-17774/Desktop/data/data/{name}/transcript.json","r") as f:
    transcript=json.load(f)

In [ ]:
transcript=list(transcript.items())

In [ ]:
i=1

In [ ]:
transcript[i]

In [ ]:
print(all_audio[i].name)
Audio(all_audio[i])

In [ ]:
urls= data_retrieval.get_video_urls_with_query('cricket in "hindi"',1)

In [ ]:
len(urls)

In [ ]:
with open("cricket_in_hindi_t.txt","w") as f:
    f.write("\n".join(urls))

In [ ]:
video_transcript={}

In [ ]:
for url in tqdm(set(urls)):
    transcript=data_retrieval.get_video_transcript(url)
    if transcript:
        video_transcript[url]=transcript

In [ ]:
len(video_transcript)

In [ ]:
valid_transcription={}

In [ ]:
for video_url,transcription in tqdm(video_transcript.items()):
    # validate_transcription(video_url,transcription)
    # print(len(transcription))
    # print(len(transcription[0]["text"].strip()))
    clean_transcript=data_validator.validate_transcription(transcription)
    if clean_transcript:
        # download_and_split_audio(video_url,clean_transcript)
        valid_transcription[video_url]=clean_transcript

In [ ]:
len(valid_transcription)

In [ ]:
with open("transt.json",'w') as f:
    json.dump(valid_transcription,f,ensure_ascii=False)

In [ ]:
root_path=Path("/home/munikumar-17774/Desktop/data/collect_data/audio")

In [ ]:
for url,transcription in tqdm(valid_transcription.items()):
    audio_processor.download_audio(url,root_path)
    

In [37]:
trans_data=valid_transcription['https://www.youtube.com/watch?v=zhtmDW7A2Yg']

In [39]:
audio_processor.split_audio_and_save_transcription("/home/munikumar-17774/Desktop/data/collect_data/audio/zhtmDW7A2Yg.mp3",trans_data)

In [38]:
audio_dir=Path("/home/munikumar-17774/Desktop/data/collect_data/audio/zhtmDW7A2Yg")

In [40]:
transcript_file=audio_dir/"transcript.json"

In [41]:
with open(transcript_file,"r") as f:
    transcript=json.load(f)

In [42]:
final_valid=[]

In [43]:
from app.nemo import NemoASR

In [44]:
asr=NemoASR()

[NeMo I 2023-11-18 16:35:48 cloud:58] Found existing object /home/munikumar-17774/.cache/torch/NeMo/NeMo_1.19.1/stt_hi_conformer_ctc_medium/be04ad0a850be76a1a79c17945fcc8cb/stt_hi_conformer_ctc_medium.nemo.
[NeMo I 2023-11-18 16:35:48 cloud:64] Re-using file from: /home/munikumar-17774/.cache/torch/NeMo/NeMo_1.19.1/stt_hi_conformer_ctc_medium/be04ad0a850be76a1a79c17945fcc8cb/stt_hi_conformer_ctc_medium.nemo
[NeMo I 2023-11-18 16:35:48 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-11-18 16:35:48 mixins:170] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2023-11-18 16:35:48 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /code/03_manifest/ulca-train-v3.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    
[NeMo W 2023-11-18 16:35:48 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /code/manifest/ulca-eval.json
    sample_rate: 16000
    batch_size: 4
    shuffle: false
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    
[NeMo W 2023-11-18 16:35:48 modelPT:174

[NeMo I 2023-11-18 16:35:48 features:291] PADDING: 0
[NeMo I 2023-11-18 16:35:49 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /home/munikumar-17774/.cache/torch/NeMo/NeMo_1.19.1/stt_hi_conformer_ctc_medium/be04ad0a850be76a1a79c17945fcc8cb/stt_hi_conformer_ctc_medium.nemo.


In [45]:
from app.models import WhisperModel

In [46]:
whisper_model=WhisperModel()

In [47]:
final_trans={}

In [48]:
for audio_name,transcription in tqdm(transcript.items()):
    audio_path=audio_dir/audio_name
    language=whisper_model.detect_language(str(audio_path))
    if language=='hi':
        text=asr.transcribe_audio(str(audio_path))[0]
        print(text,transcription)
        final_trans[audio_name]={"normal":transcription,"nemo":text}

  0%|          | 0/12 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:02<00:27,  2.54s/it]

['वन क्रिकेट का पिता किसे माना जाता है हॉबर्ट जेम्स विलियम्स टू पहला टेस्ट मैच कब और किसके बीच खेला गया'] वन क्रिकेट का पिता किस माना जाता है होबर्ट जेम्स विलियम्स तू पहले टेस्ट मैच कब और किसके बीच खेल


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

 92%|█████████▏| 11/12 [00:28<00:02,  2.49s/it]

['क्या होता है वनडे इंटरनेशनल मैचों का एक तरह का टूर्नामेंट जिसमें कई देशों की टीमें एक'] क्या होता है वनडे इंटरनेशनल माचो का एक तरह का टूर्नामेंट जिसमें कई देश के टाइम


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:30<00:00,  2.54s/it]

['दूसरे के साथ प्रतिस्पर्धा करती हैं थर्टीन टेस्ट क्रिकेट में एक ओवर में कितनी गेंदें डाली जाती'] एक दूसरे के साथ प्रतिस्पर्धा करती हैं 13 टेस्ट क्रिकेट में एक ओवर में कितनी गंदे डाली जाति है


In [49]:
with open("similarity_1.json","w") as f:
    json.dump(final_trans,f,ensure_ascii=False)